In [2]:
%run './remodelisations/communes.ipynb'
%run './remodelisations/dep.ipynb'
%run './remodelisations/regions.ipynb'
%run './remodelisations/statistiques.ipynb' # les stats ont des problèmes avec les fk -> ref vers des code de communes inconnues

In [29]:
import psycopg2 as postgres

In [30]:
BASE = 'tp_dba'
USER = 'axelmoriceau'
PSWD = ''

conn = postgres.connect(
    host='localhost',
    database=BASE,
    user=USER,
    password=PSWD,
    port='5432'
)

cur = conn.cursor()


In [31]:
tables = ['departements_cheflieu','regions_cheflieu', 'communes', 'departements', 'regions','indicateurs','statistiques']

In [32]:
# A enlever avant rendu 

drop_tables = ''
for table in tables:
    drop_tables += f'DROP TABLE IF EXISTS {table} CASCADE;'


In [33]:
cur.execute(drop_tables)
conn.commit()

In [34]:
cur.execute(open("sql/create.sql", "r").read())
conn.commit()

In [35]:
files = [
        'regions', 
        'departements',
        'communes',
        'departements_cheflieu',
        'regions_cheflieu',
        'indicateurs',
        'statistiques'
    ]

for file in files:
    path = f'./csv/filtered/{file}.csv'
    cur.copy_from(open(path, 'r'), file, sep=',', null='None')
    conn.commit()


InvalidTextRepresentation: invalid input syntax for type numeric: ""
CONTEXT:  COPY statistiques, line 35298, column valeur: ""


In [ ]:
conn.close()